In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [5]:
# Вкажіть шлях до вашої папки з даними
dataset_dir = '/kaggle/input/mstar-dataset-8-classes/Padded_imgs'

# Аугментація зображень для тренувального набору
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% даних для валідації
)

# Завантаження тренувальних зображень
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(368, 368),  # Розмір зображень
    batch_size=100,
    class_mode='categorical',  # Для багатокласової класифікації
    subset='training'  # Використовуємо підмножину для тренування
)

# Завантаження валідаційних зображень
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(368, 368),
    batch_size=100,
    class_mode='categorical',
    subset='validation'  # Використовуємо підмножину для валідації
)


Found 7576 images belonging to 8 classes.
Found 1890 images belonging to 8 classes.


In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(368, 368, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(len(train_generator.class_indices), activation='softmax'))  # Кількість класів


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30
)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1731007394.372358     115 service.cc:145] XLA service 0x798d5c005aa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731007394.372411     115 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1731007394.372416     115 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2024-11-07 19:23:31.933629: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[100,32,183,183]{3,2,1,0}, u8[0]{0}) custom-call(f32[100,64,181,181]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0

29/75 ━━━━━━━━━━━━━━━━━━━━ 2:18 3s/step - accuracy: 0.2263 - loss: 2.2337

2024-11-07 19:25:24.604387: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[76,32,183,183]{3,2,1,0}, u8[0]{0}) custom-call(f32[76,64,181,181]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-11-07 19:25:24.905484: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.30130198s
Trying algorithm eng0{} for conv (f32[76,32,183,183]{3,2,1,0}, u8[0]{0}) custom-call(f32[76,64,181,181]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_re

75/75 ━━━━━━━━━━━━━━━━━━━━ 351s 4s/step - accuracy: 0.3580 - loss: 1.7768 - val_accuracy: 0.6656 - val_loss: 0.7883
Epoch 2/30
 1/75 ━━━━━━━━━━━━━━━━━━━━ 27s 367ms/step - accuracy: 0.8100 - loss: 0.5499

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


75/75 ━━━━━━━━━━━━━━━━━━━━ 13s 171ms/step - accuracy: 0.8100 - loss: 0.5499 - val_accuracy: 0.6222 - val_loss: 1.0813
Epoch 3/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 274s 3s/step - accuracy: 0.7542 - loss: 0.6467 - val_accuracy: 0.7806 - val_loss: 0.5865
Epoch 4/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.7700 - loss: 0.5160 - val_accuracy: 0.8667 - val_loss: 0.5481
Epoch 5/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - accuracy: 0.7990 - loss: 0.5050 - val_accuracy: 0.8267 - val_loss: 0.4432
Epoch 6/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.8200 - loss: 0.4899 - val_accuracy: 0.8667 - val_loss: 0.3886
Epoch 7/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 274s 3s/step - accuracy: 0.8223 - loss: 0.4308 - val_accuracy: 0.8433 - val_loss: 0.4046
Epoch 8/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.8700 - loss: 0.3914 - val_accuracy: 0.8222 - val_loss: 0.3824
Epoch 9/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - accuracy: 0.8494 - loss: 0.3640 - val_accuracy: 0.8444 - val_loss: 

In [9]:
test_loss, test_acc = model.evaluate(validation_generator)
print('Validation accuracy:', test_acc)

19/19 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9301 - loss: 0.1766
Validation accuracy: 0.9354497194290161


In [10]:
model.save('my_model.h5')